# Hyperparameter tuning Total Moodel

# Tuning a Random Forest Regression Model

Custom built expanding window validation functions will be used 

In [4]:
import pandas as pd
import matplotlib.pyplot as plt

# Import your helper functions
from timeseries_model_utils import (
    expanding_window_splits,
    run_expanding_cv,
    expanding_window_grid_search
)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error


In [5]:
df = pd.read_csv("New_csv.csv")   # replace with your file
df = df.sort_values("Year").reset_index(drop=True)
df.head()


,Unnamed: 0,Year,Homicide,Assault,Sexual offences,Harm or endanger persons,"Robbery, blackmail, and extortion",Burglary,Theft,Fraud and related offences,...,Traffic and vehicle offences,Offences against justice procedures and orders,Offences against government,Environmental offences,Miscellaneous offences,Total,Population,Crime_Index,Total_Lagged,Population_lagged
0,0,1980,353,7208,1401,2176,426,11893,29103,19794,...,173948,27664,1376,3521,9651,331751,3112900,10.657297,331751.0,3112900.0
1,1,1981,366,7933,1092,2299,494,12974,29654,25007,...,142990,17981,2027,3360,9836,294409,3124900,9.421389,331751.0,3112900.0
2,2,1982,329,7760,1298,2231,504,14387,32682,26131,...,104739,13957,1203,4025,7878,252119,3156100,7.988308,294409.0,3124900.0
3,3,1983,407,8741,1449,2693,657,13851,32304,28481,...,105649,15332,736,2252,7774,254658,3199300,7.959804,252119.0,3156100.0
4,4,1984,406,9349,1682,2651,729,13905,33492,28069,...,116414,16389,1300,2235,8552,274022,3227100,8.491277,254658.0,3199300.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 24 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Unnamed: 0                                      45 non-null     int64  
 1   Year                                            45 non-null     int64  
 2   Homicide                                        45 non-null     int64  
 3   Assault                                         45 non-null     int64  
 4   Sexual offences                                 45 non-null     int64  
 5   Harm or endanger persons                        45 non-null     int64  
 6   Robbery, blackmail, and extortion               45 non-null     int64  
 7   Burglary                                        45 non-null     int64  
 8   Theft                                           45 non-null     int64  
 9   Fraud and related offences                   

In [7]:
target_col = "Total"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [8]:
param_grid = {
    "n_estimators": [100, 200, 400],
    "max_depth": [3, 5, None]
}

best_params, best_score, results_df = expanding_window_grid_search(
    RandomForestRegressor,
    param_grid,
    X, y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters:", best_params)
print("Best Score:", best_score)

results_df



[GridSearch] Params {'n_estimators': 100, 'max_depth': 3} → Score = 16505.2714
[GridSearch] Params {'n_estimators': 100, 'max_depth': 5} → Score = 16092.2801
[GridSearch] Params {'n_estimators': 100, 'max_depth': None} → Score = 16464.5843
[GridSearch] Params {'n_estimators': 200, 'max_depth': 3} → Score = 16184.6312
[GridSearch] Params {'n_estimators': 200, 'max_depth': 5} → Score = 16162.1387
[GridSearch] Params {'n_estimators': 200, 'max_depth': None} → Score = 16248.7641
[GridSearch] Params {'n_estimators': 400, 'max_depth': 3} → Score = 15993.6085
[GridSearch] Params {'n_estimators': 400, 'max_depth': 5} → Score = 16073.1379
[GridSearch] Params {'n_estimators': 400, 'max_depth': None} → Score = 16165.7920

Best Parameters: {'n_estimators': np.float64(400.0), 'max_depth': np.float64(3.0)}
Best Score: 15993.608491016043


,n_estimators,max_depth,score
0,100,3.0,16505.271443
1,100,5.0,16092.280086
2,100,NaN,16464.584286
3,200,3.0,16184.631155
4,200,5.0,16162.138748
5,200,NaN,16248.764143
6,400,3.0,15993.608491
7,400,5.0,16073.137898
8,400,NaN,16165.792000


In [9]:

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000]
}


In [10]:

X = X.ffill().bfill()


In [11]:
from sklearn.linear_model import Lasso

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000],
}

best_params, best_score, results_df = expanding_window_grid_search(
    Lasso,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (Lasso):", best_params)
print("Best Score:", best_score)
results_df


[GridSearch] Params {'alpha': 0.001, 'max_iter': 1000} → Score = 20931.6176


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.190e+09, tolerance: 1.473e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.906e+09, tolerance: 1.953e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.100e+09, tolerance: 2.3

[GridSearch] Params {'alpha': 0.001, 'max_iter': 5000} → Score = 20931.6306
[GridSearch] Params {'alpha': 0.001, 'max_iter': 10000} → Score = 20931.6306
[GridSearch] Params {'alpha': 0.01, 'max_iter': 1000} → Score = 20931.6096


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.589e+08, tolerance: 1.953e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.613e+09, tolerance: 2.376e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.551e+09, tolerance: 2.4

[GridSearch] Params {'alpha': 0.01, 'max_iter': 5000} → Score = 20931.6225
[GridSearch] Params {'alpha': 0.01, 'max_iter': 10000} → Score = 20931.6225
[GridSearch] Params {'alpha': 0.1, 'max_iter': 1000} → Score = 20931.5294


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.250e+09, tolerance: 2.440e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.547e+09, tolerance: 2.440e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.037e+09, tolerance: 2.4

[GridSearch] Params {'alpha': 0.1, 'max_iter': 5000} → Score = 20931.5419
[GridSearch] Params {'alpha': 0.1, 'max_iter': 10000} → Score = 20931.5419
[GridSearch] Params {'alpha': 1, 'max_iter': 1000} → Score = 20930.7266


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.620e+09, tolerance: 2.453e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.731e+09, tolerance: 2.734e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.449e+09, tolerance: 3.1

[GridSearch] Params {'alpha': 1, 'max_iter': 5000} → Score = 20930.7352
[GridSearch] Params {'alpha': 1, 'max_iter': 10000} → Score = 20930.7352
[GridSearch] Params {'alpha': 10, 'max_iter': 1000} → Score = 20922.6686
[GridSearch] Params {'alpha': 10, 'max_iter': 5000} → Score = 20922.6686
[GridSearch] Params {'alpha': 10, 'max_iter': 10000} → Score = 20922.6686
[GridSearch] Params {'alpha': 50, 'max_iter': 1000} → Score = 20886.8171
[GridSearch] Params {'alpha': 50, 'max_iter': 5000} → Score = 20886.8171
[GridSearch] Params {'alpha': 50, 'max_iter': 10000} → Score = 20886.8171
[GridSearch] Params {'alpha': 100, 'max_iter': 1000} → Score = 20842.0017
[GridSearch] Params {'alpha': 100, 'max_iter': 5000} → Score = 20842.0017
[GridSearch] Params {'alpha': 100, 'max_iter': 10000} → Score = 20842.0017

Best Parameters (Lasso): {'alpha': np.float64(100.0), 'max_iter': np.float64(1000.0)}
Best Score: 20842.001703861297


,alpha,max_iter,score
0,0.001,1000,20931.617567
1,0.001,5000,20931.630609
2,0.001,10000,20931.630609
3,0.010,1000,20931.609570
4,0.010,5000,20931.622542
5,0.010,10000,20931.622542
6,0.100,1000,20931.529441
7,0.100,5000,20931.541874
8,0.100,10000,20931.541874
9,1.000,1000,20930.726561


# train a lasso with alpha 0.001 and max iter 1000 and save it

In [13]:
from sklearn.linear_model import Lasso

# Initialize model
lasso = Lasso(alpha=100, max_iter=1000)

# Train model
lasso.fit(X, y)


Lasso(alpha=100)

In [14]:
import joblib

joblib.dump(lasso, "Lasso_last.joblib")

['Lasso_last.joblib']

In [15]:
lasso.predict(X[-1:])

array([215634.32445086])

In [16]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


# Train Random forest & Save it

In [18]:
rd = RandomForestRegressor(n_estimators=400, max_depth=None)

rd.fit(X, y)

RandomForestRegressor(n_estimators=400)

In [19]:
X[-1:]

,Total_Lagged,Year,Population_lagged
44,216533.0,2024,5223100.0


In [20]:
rd.predict(X[-1:])

array([218376.42])

In [21]:
joblib.dump(rd, "RandomForest_last.joblib")

['RandomForest_last.joblib']

# Tuning a Gradient Boosting Algorithm

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    "n_estimators": [100, 300, 600],          # number of boosting stages
    "learning_rate": [0.01, 0.05, 0.1],       # shrinkage
    "max_depth": [1,2, 3, 4],                   # depth of each tree (via base estimator)
                    # only used when loss="huber" or "quantile"
}

best_params, best_score, results_df = expanding_window_grid_search(
    GradientBoostingRegressor,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (GBR):", best_params)
print("Best Score:", best_score)
results_df

[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 1} → Score = 25140.8197
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 2} → Score = 19677.4502
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 3} → Score = 18373.3148
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.01, 'max_depth': 4} → Score = 18951.0361
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 1} → Score = 16147.6991
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 2} → Score = 13726.3502
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 3} → Score = 14940.7726
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 4} → Score = 16317.5513
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 1} → Score = 15024.2094
[GridSearch] Params {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 2} → S

,n_estimators,learning_rate,max_depth,score
0,100,0.01,1,25140.819700
1,100,0.01,2,19677.450243
2,100,0.01,3,18373.314768
3,100,0.01,4,18951.036149
4,100,0.05,1,16147.699092
5,100,0.05,2,13726.350203
6,100,0.05,3,14940.772617
7,100,0.05,4,16317.551350
8,100,0.10,1,15024.209401
9,100,0.10,2,13562.050006


In [24]:

import joblib


gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

# Train the model
gbr.fit(X, y)


GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

In [25]:
joblib.dump(gbr, "total.joblib")


['total.joblib']

# Hyperparameter Tuning the Population Model

In [27]:
target_col = "Population"        # or any column you want to predict
feature_cols = ["Total_Lagged","Year","Population_lagged"]

X = df[feature_cols]
y = df[target_col]


In [28]:
from sklearn.linear_model import Lasso

param_grid = {
    "alpha": [0.001, 0.01, 0.1, 1, 10, 50, 100],
    "max_iter": [1000, 5000, 10000],
}

best_params, best_score, results_df = expanding_window_grid_search(
    Lasso,
    param_grid,
    X,
    y,
    initial_train_size=10,
    horizon=1,
    step=1
)

print("\nBest Parameters (Lasso):", best_params)
print("Best Score:", best_score)
results_df


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.744e+09, tolerance: 3.056e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.097e+09, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.580e+09, tolerance: 3.8

[GridSearch] Params {'alpha': 0.001, 'max_iter': 1000} → Score = 28869.4840
[GridSearch] Params {'alpha': 0.001, 'max_iter': 5000} → Score = 28869.1854
[GridSearch] Params {'alpha': 0.001, 'max_iter': 10000} → Score = 28869.1854
[GridSearch] Params {'alpha': 0.01, 'max_iter': 1000} → Score = 28869.4769


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.396e+08, tolerance: 3.056e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.090e+09, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.986e+09, tolerance: 3.8

[GridSearch] Params {'alpha': 0.01, 'max_iter': 5000} → Score = 28869.1784
[GridSearch] Params {'alpha': 0.01, 'max_iter': 10000} → Score = 28869.1784
[GridSearch] Params {'alpha': 0.1, 'max_iter': 1000} → Score = 28869.4063


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.395e+08, tolerance: 3.455e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.369e+09, tolerance: 3.871e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.439e+09, tolerance: 4.3

...

C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.796e+09, tolerance: 5.777e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.995e+09, tolerance: 6.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.673e+09, tolerance: 6.8

[GridSearch] Params {'alpha': 50, 'max_iter': 1000} → Score = 28830.7231
[GridSearch] Params {'alpha': 50, 'max_iter': 5000} → Score = 28830.4724
[GridSearch] Params {'alpha': 50, 'max_iter': 10000} → Score = 28830.4724


C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.417e+08, tolerance: 5.777e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.183e+09, tolerance: 6.290e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.742e+09, tolerance: 6.8

[GridSearch] Params {'alpha': 100, 'max_iter': 1000} → Score = 28791.9774
[GridSearch] Params {'alpha': 100, 'max_iter': 5000} → Score = 28791.7789
[GridSearch] Params {'alpha': 100, 'max_iter': 10000} → Score = 28791.7789

Best Parameters (Lasso): {'alpha': np.float64(100.0), 'max_iter': np.float64(5000.0)}
Best Score: 28791.778854599088


,alpha,max_iter,score
0,0.001,1000,28869.484035
1,0.001,5000,28869.185399
2,0.001,10000,28869.185399
3,0.010,1000,28869.476877
4,0.010,5000,28869.178433
5,0.010,10000,28869.178433
6,0.100,1000,28869.406332
7,0.100,5000,28869.108751
8,0.100,10000,28869.108751
9,1.000,1000,28868.709635


In [29]:
gbr = GradientBoostingRegressor(
    n_estimators=600,
    learning_rate=0.1,
    max_depth=1,
    random_state=42
)

In [30]:
lasso_model = Lasso(alpha = 0.001,max_iter = 1000)

In [31]:
lasso_model.fit(X,y)

C:\Users\hp\miniconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.573e+10, tolerance: 1.896e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.001)

In [32]:
joblib.dump(lasso_model, "population.joblib")

['population.joblib']

# Recursive Model Function

In [34]:
def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    scaler_X_pop,
    scaler_X_total,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    scaler_X_pop : fitted scaler
        Scaler used for population model
    scaler_X_total : fitted scaler
        Scaler used for total model
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        # ---- Predict Population_lagged ----
        X_pop_scaled = scaler_X_pop.transform(X_current)
        pop_pred = population_model.predict(X_pop_scaled)[0]

        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        X_total_scaled = scaler_X_total.transform(X_current)
        total_pred = total_model.predict(X_total_scaled)[0]

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # ---- Shift lags (if needed) ----
        # Example: Population_lagged_1 → Population_lagged_2
        for col in X_current.columns:
            if "_lag" in col:
                lag_num = int(col.split("_lag")[-1])
                if lag_num > 1:
                    X_current[col.replace(f"_lag{lag_num}", f"_lag{lag_num-1}")] = X_current[col]

        # Set newest lag
        X_current[population_feature_name] = pop_pred

    return pd.DataFrame(forecasts)


In [35]:
import pandas as pd

def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        # ---- Predict Population_lagged ----
        pop_pred = population_model.predict(X_current)[0]

        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        total_pred = total_model.predict(X_current)[0]

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # ---- Shift lagged features ----
        for col in X_current.columns:
            if "_lag" in col:
                lag_num = int(col.split("_lag")[-1])
                if lag_num > 1:
                    X_current[col.replace(
                        f"_lag{lag_num}", f"_lag{lag_num-1}"
                    )] = X_current[col]

        # Set newest lag explicitly
        X_current[population_feature_name] = pop_pred

    return pd.DataFrame(forecasts)


In [36]:
gbr.fit(X,y)

GradientBoostingRegressor(max_depth=1, n_estimators=600, random_state=42)

In [37]:
predictions = recursive_forecast_with_exogenous(X[:-1], lasso_model, gbr, "Population_lagged")

ValueError: invalid literal for int() with base 10: 'ged'

# Fix the  error

In [43]:
import pandas as pd

def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.

    Parameters
    ----------
    X_last : pd.DataFrame
        Last known row of features (t)
    population_model : trained model
        Model that predicts Population_lagged
    total_model : trained model
        Model that predicts total
    population_feature_name : str
        Column name of Population_lagged
    horizon : int
        Number of recursive steps

    Returns
    -------
    pd.DataFrame
        Forecasted Population and Total values
    """

    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):

        print(X_current)
        # ---- Predict Population_lagged ----
        pop_pred = population_model.predict(X_current)[0]
        print(pop_pred)
        # Update population feature
        X_current[population_feature_name] = pop_pred

        # ---- Predict total ----
        total_pred = total_model.predict(X_current)[0]

        

        # Store predictions
        forecasts.append({
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # # ---- Shift lagged features ----
        # for col in X_current.columns:
        #     if "_lag" in col:
        #         lag_num = int(col.split("_lag")[-1])
        #         if lag_num > 1:
        #             X_current[col.replace(
        #                 f"_lag{lag_num}", f"_lag{lag_num-1}"
        #             )] = X_current[col]

        # # Set newest lag explicitly
        # X_current[population_feature_name] = pop_pred
        X_current[population_feature_name] = pop_pred
        X_current["Total_Lagged"] = total_pred
        X_current["Year"] += 1 

    return pd.DataFrame(forecasts)

In [45]:
predictions = recursive_forecast_with_exogenous(X[-1:], lasso_model, gbr, "Population_lagged")

    Total_Lagged  Year  Population_lagged
44      216533.0  2024          5223100.0
5277296.62856452
    Total_Lagged  Year  Population_lagged
44  5.266385e+06  2025       5.277297e+06
3362451.7015081774
    Total_Lagged  Year  Population_lagged
44  4.318596e+06  2026       3.362452e+06
2083267.5879166853


In [47]:
predictions

,step,Population_lagged_pred,total_pred
0,1,5.277297e+06,5.266385e+06
1,2,3.362452e+06,4.318596e+06
2,3,2.083268e+06,4.136292e+06


In [ ]:
X

In [49]:
def get_last_known_data(data_file="New_csv.csv"):
    """Get the last known data point from the dataset"""
    try:
        df = pd.read_csv(data_file)
        df = df.sort_values("Year").reset_index(drop=True)
        
        # Extract features needed for prediction
        feature_cols = ["Total_Lagged", "Year", "Population_lagged"]
        X_last = df[feature_cols].iloc[-1:].copy()
        last_year = int(df.iloc[-1]["Year"])
        
        return X_last, last_year
    except FileNotFoundError:
        raise Exception(f"Dataset file '{data_file}' not found!")
    except Exception as e:
        raise Exception(f"Error loading data: {e}")

In [51]:
result = get_last_known_data()

In [57]:
result

(    Total_Lagged  Year  Population_lagged
 44      216533.0  2024          5223100.0,
 2024)

In [99]:
def recursive_forecast_with_exogenous(
    X_last: pd.DataFrame,
    population_model,
    total_model,
    population_feature_name: str,
    horizon: int = 3
):
    """
    Recursive forecast where Population_lagged is predicted first
    and then used to predict total.
    """
    X_current = X_last.copy()
    forecasts = []

    for step in range(1, horizon + 1):
        # Predict Population_lagged
        print(population_model.predict(X_current))
        pop_pred = population_model.predict(X_current)
        
        
        # Update population feature
        X_current[population_feature_name] = pop_pred

        # Predict total
        total_pred = total_model.predict(X_current)

        # Store predictions
        current_year = int(X_current["Year"].iloc[0]) 
        forecasted_year = current_year+1
        forecasts.append({
            "Year": forecasted_year,
            "step": step,
            "Population_lagged_pred": pop_pred,
            "total_pred": total_pred
        })

        # Update features for next iteration
        X_current[population_feature_name] = pop_pred
        X_current["Total_Lagged"] = total_pred
        X_current["Year"] = current_year + 1

    return pd.DataFrame(forecasts)


In [101]:
def predict_total_for_year_recursive(target_year: int, data_file="New_csv.csv"):
    """
    Predict total crime value for a specific year using recursive forecasting.
    
    Args:
        target_year (int): The year to predict total values for
        data_file (str): Path to the dataset file containing lagged features
        
    Returns:
        float: Predicted total value for the target year
    """
    # if not recursive_models_loaded:
    #     if not load_recursive_models():
    #         raise Exception("Failed to load recursive models. Ensure population.joblib and total.joblib exist.")
    
    # Get last known data
    X_last, last_year = get_last_known_data(data_file)
    
    # Validate target year
    if target_year <= last_year:
        raise Exception(f"Target year ({target_year}) must be greater than last known year ({last_year})")
    
    # Calculate forecast horizon
    horizon = target_year - last_year
    
    # Perform recursive forecast
    forecasts = recursive_forecast_with_exogenous(
        X_last=X_last,
        population_model=lasso_model,
        total_model=gbr,
        population_feature_name="Population_lagged",
        horizon=horizon
    )
    print(forecasts)
    
    # Get the prediction for the target year
    #print(forecasts[forecasts["Year"] == target_year]["total_pred"])
    target_prediction = forecasts[forecasts["Year"] == target_year]["total_pred"].iloc[0]
    
    
    return float(target_prediction)

In [103]:
result = predict_total_for_year_recursive(2025)

[5277296.62856452]
   Year  step Population_lagged_pred           total_pred
0  2025     1     [5277296.62856452]  [5266385.188041851]


C:\Users\hp\AppData\Local\Temp\ipykernel_20400\765497636.py:40: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(target_prediction)
